In [1]:
import mysql.connector
import pandas as pd
import numpy as np 
import timeit
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from imblearn.over_sampling import ADASYN
import os
model_dir = os.getcwd()

In [2]:
def fetch_df(db,table):
    '''fetch table from database'''
    try:
        connection = mysql.connector.connect(host='localhost',database='bid',user='root',password='yining610')
        sql_select_Query = "select * from " + db + "." + table
        cursor = connection.cursor()
        cursor.execute(sql_select_Query)
        # get all records
        records = cursor.fetchall()
        print("Total number of rows in table: ", cursor.rowcount)
        return records
    except mysql.connector.Error as e:
        print("Error reading data from MySQL table", e)
    finally:
        if connection.is_connected():
            connection.close()
            cursor.close()
            print("MySQL connection is closed")


def vectorized_input(df):
    # For ipinyou data, has to be changed when applying to the real data
    categorical_fields = ['weekday', 'hour', 'logtype',
                          'useragent', 'region', 'city', 'adexchange',
                          'slotwidth', 'slotheight',
                          'slotvisibility', 'slotformat', 'slotprice', 'creative',
                          'keypage', 'advertiser']

    one_hot_list = []
    # each element represent a sparse matrix with size (#observations, onehot length)
    for categorical_field in categorical_fields:
        enc = OneHotEncoder()
        one_hot_list.append(enc.fit_transform(np.array(df[categorical_field]).reshape(-1,1)))
    output = np.array(df.click)

    # Concate all the variables row-wise
    fields = list(sub1+sub2+sub3+sub4+sub5+sub6+sub7+sub8+sub9+sub10+sub11+sub12+sub13+sub14+sub15 for
             sub1,sub2,sub3,sub4,sub5,sub6,sub7,sub8,sub9,sub10,sub11,sub12,sub13,sub14,sub15 in 
             zip(one_hot_list[0].toarray().tolist(),
                 one_hot_list[1].toarray().tolist(),
                 one_hot_list[2].toarray().tolist(),
                 one_hot_list[3].toarray().tolist(),
                 one_hot_list[4].toarray().tolist(),
                 one_hot_list[5].toarray().tolist(),
                 one_hot_list[6].toarray().tolist(),
                 one_hot_list[7].toarray().tolist(),
                 one_hot_list[8].toarray().tolist(),
                 one_hot_list[9].toarray().tolist(),
                 one_hot_list[10].toarray().tolist(),
                 one_hot_list[11].toarray().tolist(),
                 one_hot_list[12].toarray().tolist(),
                 one_hot_list[13].toarray().tolist(),
                 one_hot_list[14].toarray().tolist()))
                 
    fields = np.matrix(fields)
    output.shape += (1, )
    return fields,output


def evaluate_recall(truth, predictions):  
    recall = recall_score(truth, predictions.round())
    return ('recall', recall, True)

In [3]:
data = fetch_df('bid','train_data')
df = pd.DataFrame(data,columns = ['click','weekday','hour','bidid','timestamp','logtype','ipinyouid',
                                  'useragent','IP','region','city','adexchange','domain','url','urlid',
                                  'slotid','slotwidth','slotheight','slotvisibility','slotformat','slotprice',
                                  'creative','bidprice','payprice','keypage','advertiser','usertag'])
df.usertag = df.usertag.str.rstrip('\r')

Total number of rows in table:  1048575
MySQL connection is closed


In [4]:
# fields,output = vectorized_input(df[0:200000])
fields,output = vectorized_input(df)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(fields, output, test_size=0.3,random_state=42,stratify=output)

In [6]:
ada = ADASYN(random_state=0,sampling_strategy=0.6)
X_train,y_train = ada.fit_resample(X_train,y_train) # minority / majority = 0.6 after resampling

C:\Users\610\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [25]:
y_train = y_train.reshape(y_train.shape[0],1)

In [16]:
sum(y_train == 1)

440176

In [17]:
sum(y_test == 1)

array([222])

In [27]:
# Customize layers
class Linear(keras.layers.Layer):
    def __init__(self, p):
        super(Linear, self).__init__()
        # bias and weights
        self.w0 = tf.Variable(tf.zeros([1]),trainable=True,dtype=tf.float32)
        self.W = tf.Variable(tf.zeros([p]),trainable=True,dtype=tf.float32)
    def call(self, inputs):
        return tf.add(self.w0,tf.reduce_sum(tf.multiply(self.W,inputs),axis=1,keepdims=True))

class Latent(keras.layers.Layer):
    def __init__(self,k,p):
        super(Latent, self).__init__()
        #latent(interaction) vector, to measure the impact of interactions with other features
        self.V = tf.Variable(tf.random.normal(shape=[k,p]),trainable=True,dtype=tf.float32)

    def call(self, inputs):
        return tf.matmul(inputs, tf.transpose(self.V))

class Interaction(Latent):
    def __init__(self,latent_layer):
        super(Interaction, self).__init__(k,p)
        self.V = latent_layer.V
        
    def call(self, inputs):
        return tf.multiply(0.5,
                    tf.reduce_sum(
                        tf.subtract(
                            tf.pow(tf.matmul(inputs, tf.transpose(self.V)), 2),
                            tf.matmul(tf.pow(inputs, 2), tf.transpose(tf.pow(self.V, 2)))),
                        1, keepdims=True))

In [28]:
# initial_bias = np.log(sum(y_train==1) / (y_train.shape[0]-sum(y_train==1))) # Introduce bias 
# output_bais = tf.keras.initializers.Constant(initial_bias)
# FM Component
n,p = X_train.shape
# number of latent factors 
k = 10

X = keras.Input(shape=(p),name='Field')

latent_layer = Latent(k,p)
X_latent = latent_layer(X)

linear_layer = Linear(p)
linear_term = linear_layer(X)

interaction_layer = Interaction(latent_layer)
interaction_term = interaction_layer(X)

y_FM = layers.Dense(1,activation='sigmoid')(tf.add(linear_term,interaction_term))

# Deep Component
X_deep = layers.Dense(400, activation='relu',name='First_Layer_DNN')(X_latent)
X_deep = layers.Dropout(0.7)(X_deep)
X_deep = layers.Dense(400, activation='relu', name='Second_Layer_DNN')(X_deep)
X_deep = layers.Dropout(0.7)(X_deep)
X_deep = layers.Dense(400, activation='relu', name='Third_Layer_DNN')(X_deep)
y_DNN = layers.Dense(1,activation='sigmoid', name='DNN_output')(X_deep)

yhat = layers.Dense(1,activation='sigmoid',name='yhat')(tf.math.add(y_FM,y_DNN))

model = Model(inputs = X, outputs = yhat)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Field (InputLayer)             [(None, 967)]        0           []                               
                                                                                                  
 latent_1 (Latent)              (None, 10)           9670        ['Field[0][0]']                  
                                                                                                  
 First_Layer_DNN (Dense)        (None, 400)          4400        ['latent_1[0][0]']               
                                                                                                  
 dropout_2 (Dropout)            (None, 400)          0           ['First_Layer_DNN[0][0]']        
                                                                                            

In [29]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]



early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_recall', 
    verbose=1,
    patience=5,
    mode='max',
    restore_best_weights=True)


In [30]:
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=METRICS)
logs = model.fit(X_train, 
                y_train, 
                epochs=100, 
                batch_size=64, 
                shuffle=True,
                validation_data=(X_test, y_test),
                validation_split = 0.2, 
                validation_freq = 1, 
                callbacks=[early_stopping])

Epoch 1/100
14671/14671 [==============================] - 186s 13ms/step - loss: 0.4441 - tp: 72317.0000 - fp: 22779.0000 - tn: 710704.0000 - fn: 133127.0000 - accuracy: 0.8340 - precision: 0.7605 - recall: 0.3520 - auc: 0.7022 - prc: 0.5446 - val_loss: 1.4116 - val_tp: 74459.0000 - val_fp: 0.0000e+00 - val_tn: 0.0000e+00 - val_fn: 160273.0000 - val_accuracy: 0.3172 - val_precision: 1.0000 - val_recall: 0.3172 - val_auc: 0.0000e+00 - val_prc: 1.0000
Epoch 2/100
14671/14671 [==============================] - 189s 13ms/step - loss: 0.2927 - tp: 131531.0000 - fp: 10963.0000 - tn: 722520.0000 - fn: 73913.0000 - accuracy: 0.9096 - precision: 0.9231 - recall: 0.6402 - auc: 0.8413 - prc: 0.7761 - val_loss: 1.3781 - val_tp: 106970.0000 - val_fp: 0.0000e+00 - val_tn: 0.0000e+00 - val_fn: 127762.0000 - val_accuracy: 0.4557 - val_precision: 1.0000 - val_recall: 0.4557 - val_auc: 0.0000e+00 - val_prc: 1.0000
Epoch 3/100
14671/14671 [==============================] - 189s 13ms/step - loss: 0.2000 

KeyboardInterrupt: 

In [ ]:
model.save('DeepFM_V2')

AssertionError: Tried to export a function which references 'untracked' resource Tensor("1811471:0", shape=(), dtype=resource). TensorFlow objects (e.g. tf.Variable) captured by functions must be 'tracked' by assigning them to an attribute of a tracked object or assigned to an attribute of the main object directly.

 Trackable Python objects referring to this tensor (from gc.get_referrers, limited to two hops):
<tf.Variable 'Variable:0' shape=(967,) dtype=float32>